# SIATEC Patterns and Tune Families in The Session

### Abstract

In this notebook we will run one of the best-known musical pattern extraction algorithms on tunes from *The Session*, and implement a simple method for the task of tune family detection (similar to the task of cover-song detection).


### Introduction

As already described (Polifonia deliverables D3.1-D3.6, 2021-2024; Danny Diamond MSc thesis, 2024), the concept of tune families is an important one in the musicology of traditional music, and we have developed methods of automatically detecting tune families based on several types of $n$-gram melodic patterns. The goal of this notebook is to implement another method, based on a totally different definition of melodic patterns. As we will see, this method does not work well -- not because of a weakness in the algorithms or the patterns, but because this new definition of pattern has a different goal -- *compression* rather than *linking*.


### SIATEC patterns

SIATEC, COSIATEC, and SIATECCompress are members of a family of pattern-extraction algorithms developed by David Meredith in multiple papers over several years. A representative paper is: [*COSIATEC and SIATECCompress:
Pattern discovery by geometric compression*](https://vbn.aau.dk/en/publications/094ac3cf-78ca-4f87-8c6a-77303e3b1d36), David Meredith, Music Information Retrieval Evaluation eXchange (MIREX 2013).

These algorithms work on *point-set* data, ie music where each note is represented only by an onset time (in beats, or MIDI ticks, or another unit) and a pitch (chromatic or diatonic). The algorithms work by detecting subsets of the data which occur more than once. They aim to extract the best subsets, which occur most often, using heuristics such as "compactness" of the pattern, to help. When detected, the pattern is removed from the dataset, and each repetition is replaced by a reference. Each repetition that is detected and encoded in such an extracted pattern thus allows the original dataset to be compressed.


We focus in this notebook on the SIATECCompress algorithm, which allows patterns to overlap with each other. This choice is primarily because it is faster (Meredith, 2013) and our corpus is large.

The implemention we use is `OMNISIA.jar`, from: http://www.titanmusic.com/software.php. We are grateful to David Meredith for providing a high-quality implementation.

In [1]:
import os, subprocess
from pathlib import Path
from collections import defaultdict
import pandas as pd
import time
import pickle

### Pre-processing

We begin with the file [`thesession.abc`, available in GitHub](https://github.com/adactio/TheSession-data). For certainty and comparability, we host a specific dump under the Polifonia project:

https://github.com/polifonia-project/folk_ngram_analysis/blob/master/FoNN/thesession_corpus/abc/thesession.abc

We process this to MIDI files using `abc2midi.exe`, available from: https://ifdo.ca/~seymour/runabc/top.html 

In particular, https://ifdo.ca/~seymour/runabc/abcMIDI-2024.03.21.zip has `abc2midi.exe`.

The command line we use is:

`.\abc2midi.exe thesession.abc -silent -NGRA -NCOM`

This gives a `.mid` file for each tune in the `.abc` - approximately 40,000.

We then run the SIATEC algorithm on each MIDI file as follows. We have a function to run the algorithm once for a single MIDI file and a function to loop over all MIDI files, and a function to parse the output of a single run to a new .pat file.

In [18]:
def gather_patterns(lines):
    '''parses a .SIATECCompress file or similar. each pattern is on a line by itself
    in a format like:
    
    T(P(p(721,29),p(16081,29),p(31441,29),p(46801,29)),V(v(0,0),v(240,0),v(6480,2),v(7680,0),v(7920,0),v(9120,0),v(10320,2),v(10560,3),v(10800,4),v(11040,5),v(11280,6),v(11520,4),v(11760,3),v(12000,1),v(12240,2),v(12480,0),v(12720,-1),v(12960,-3),v(13200,-5),v(13440,-4),v(13680,-3),v(13920,-5),v(14160,-4)))

    and we will convert to the following format:
    
    (pattern, n_occurrences, occurences, length, complexity)
    
    where:
    
    pattern is a list of (time, pitch) pairs, starting at (0, 0) because we translate for comparability
    occurrences is a list of time values
    n_occurrences is the length of that list
    length is the number of pairs in pattern
    complexity is the number of unique pitches in the pattern, divided by its length
    
    ([(0, 0), (15360, 0), (30720, 0), (46080, 0)], 23, [0, 240, 6480, 7680, 7920, 9120, 10320, 10560, 10800, 11040, 11280, 11520, 11760, 12000, 12240, 12480, 12720, 12960, 13200, 13440, 13680, 13920, 14160], 4, 0.25)

    The MIDI time values look strange - but they are set algorithmically by abc2midi, 
    so corresponding notes should end up on corresponding time values.
    '''
    
    # these little fns are to help eval() to parse a pattern line (output by OMNISIA) by itself
    def T(P, V):
        return P, V
    def P(*ps):
        return ps
    def V(*vs):
        return vs
    def p(time, note):
        return (time, note)
    def v(time, note):
        return (time, note)
    for line in lines:
        # each pattern is in a line by itself
        # when we hit a blank line (length 1), there are no more patterns, so break
        if len(line) == 1: break
            
        # otherwise parse line with eval
        pattern, occurrences = eval(line)
        
        # we could set a threshold, ie minimum number of occurrences of pattern per tune,
        # but we haven't done that in other work, so we will not do it here
        # if len(occurrences) < 2: continue 
        
        # some basic stats
        length = len(pattern)
        complexity = len(set(n[1] for n in pattern)) / length
        
        # we normalise patterns so that all patterns start at (0, 0)
        # bearing in mind we use diatonic pitch.
        time0, pitch0 = pattern[0]
        pattern = [(pi[0] - time0, pi[1] - pitch0) for pi in pattern]
        locations = [occ[0] for occ in occurrences]
        #print('pattern', pattern)
        #print('locations', locations)
        #print('length', length)
        #print('complexity', complexity)
        
        yield pattern, len(locations), locations, length, complexity

In [17]:
def run_omnisia(filename, algo='SIATECCompress'):
    '''algo can be SIATECCompress (faster, overlapping patterns) or COSIATEC'''
    
    # we have to munge the filenames a bit:
    base = Path(filename).stem
    sufx = Path(filename).suffixes[0][1:]
    
    if algo == 'SIATECCompress': suffix = 'SIATECCompress'
    elif algo == 'COSIATEC': suffix = 'cos'
    else: raise ValueError(algo)
        
    output = f'test/{base}-{sufx}/{base}-diat.{suffix}'

    # -d true: morphetic (diatonic) pitch and so the filename below is 'diat', not 'chrom'
    # -o test: output directory
    # -nodate true: don't store date, avoid making unneeded directories
    # -min 3: no patterns less than 3
    cmd = f'java -Xmx1024M -jar OMNISIA.jar -a {algo} -d true -o test -nodate true -min 3 -i {filename}'.split(' ')
    print('running OMNISIA, this may take a few seconds...')
    subprocess.check_output(cmd)

    # we read the log file:
    # print('reading ' + output)
    L = open(output).readlines()
    return L

In [19]:
def run_omnisia_all(dirname):
    '''loop over MIDI files, run OMNISIA, and read its output, and write to .pat file'''
    i = 0
    start = time.time()
    failed = []
    for midiname in os.listdir(dirname):
        if not midiname.endswith('.mid'): continue
        patternname = midiname.replace('.mid', '.pat')
        
        # only run if the output doesn't exist
        # this is to help us run in batches, stop and restart
        if Path(os.path.join(dirname, patternname)).is_file(): continue 
           
        try:
            print('trying', midiname)
            L = run_omnisia(os.path.join(dirname, midiname))
            f = open(os.path.join(dirname, patternname), 'w')
            for pat in gather_patterns(L):
                f.write(pat+'\n')
            f.close()
        except:
            # an exception can arise such as an out of heap space memory error
            print(midiname, 'failed')
            failed.append(midiname) # just pass, try this file again later
        i += 1
        print(f'{i}: {midiname}: time {time.time() - start:.1f}s')
    return failed

In [50]:
# uncomment the line below to run OMNISIA on all .mid files - this will take hours or days
# failed = run_omnisia_all('thesession_dir')


A small number of `.mid` files are not processed correctly. It's likely that a larger heap space or some pre-processing could solve this issue. For now we can ignore it, as the number is very small relative to the size of the corpus.

In [ ]:
len(failed)

Next we read all the pat files, parsing them into a single data structure. We have a function that works for a single output (from a single tune), and a function that loops over all tunes.

In [24]:
def read_all_pat_files(dirname, test_sample=None):
    i = 0
    d = {}
    start = time.time()
    for patternname in os.listdir(dirname):
        if patternname.endswith('.pat'): 
            id = int(patternname[10:-4])
            # print(patternname, id)
            if id in d: continue
            try:
                d[id] = read_patterns(os.path.join(dirname, patternname))
            except ValueError:
                pass
            i += 1
        if test_sample is not None and i > test_sample:
            break
    return d

In [26]:
def read_patterns(filename):
    """Read a .pat file, which contains one pattern per line, already parsed"""
    lines = open(filename).readlines()
    results = []
    for line in lines:
        try:
            # a hack - in an old version of the code we saved just (pattern, n_occurrences)
            # but in the current version, we save (pattern, n_occurrences, occurrences, pattern_len, pattern_complexity)
            # Only (pattern, n_occurrences) are needed for our current project. To avoid a large re-run
            # we code this function to accept either format.
            pattern, n_occurrences, occurrences, pattern_len, pattern_complexity = eval(line)
        except:
            pattern, n_occurrences = eval(line)
        results.append((pattern, n_occurrences))
    return results

In [ ]:
# uncomment this line to read in all the .pat files to a data structure
# d, failed = read_omnisia_outputs('thesession_dir')

# and save to a pkl
# f = open('data/thesession_omnisia_patterns.pkl', 'wb')
# pickle.dump(d, f)

Above, in two places, we have commented-out lines which launch large runs. Instead, for this notebook, the user can load the resulting data directly as a pickle:

In [ ]:
f = open('data/thesession_omnisia_patterns.pkl', 'rb')
d = pickle.load(f)

The pattern data is not very large in memory:

In [54]:
import sys
sys.getsizeof(d)

1310800

### Part 2: using patterns for tune family detection

Next, we can proceed to use our pattern data `d` for tune family detection.

We begin with ground truth, a set of tune family annotations described by Danny Diamond (Deliverables D5.5-5.6, and MSc thesis, 2024). The tunes in these annotations are from The Session, so their patterns have already been processed above.

In [51]:
annotated = pd.read_csv('data/thesession_subset_metadata_with_tune_family_annotation.csv')

There are 315 members in 10 tune families:

In [52]:
len(annotated)

315

In [54]:
set(annotated['tune_family'])

{'Blackbird',
 'Drowsy Maggie',
 'Gilderoy',
 "Greig's Pipes",
 'Hob or Nob',
 "Jenny's Welcome to Charlie",
 'Johnny Cope',
 "Lord McDonald's",
 "O'Sullivan's March",
 'Road to Lisdoonvarna'}

## Tune similarity via patterns



In [14]:
def get_pats(tune):
    '''Given a tune represented by a list of (pattern, n_occurrences) values,
    take only the non-trivial patterns and return them as a set.'''
    s = set()
    for pat, n_occurrences in tune:
        # complexity = len(set(el[1] for el in pat)) / len(pat)
        unique_notes = len(set(el[1] for el in pat))
        if unique_notes > 1:
            s.add(tuple(pat))
    return s
    
def compare_patterns(tune_i, tune_j):
    '''Check for an intersection among the non-trivial patterns of two tunes
    (each represented by a list of (pattern, n_occurrences) values as above.)'''
    pi = get_pats(tune_i)
    pj = get_pats(tune_j)
    intersect = pi.intersection(pj)
    if intersect: return True
    else: return False

Next we will go through all of the 315 annotated tune-family members, and for each of them
we'll find which of the 40k The Session tunes it has any patterns in common with. 

In [36]:
annotated_ids = list(annotated['identifiers'])

In [ ]:
links = defaultdict(list)
k = 0
for i in annotated_ids: # just the 315 annotated tune-family members
    print(f'working on annotated id {i} ({k}/{len(annotated_ids)})')
    for j in d: # all 40k The Session members
        if i == j: continue
        if compare_patterns(d[i], d[j]):
            links[i].append(j)
    k += 1


Next, our method is to calculate the degree of similarity as *intersection divided by union* (IOU) between two sets of patterns.

In [38]:
def intersection_over_union(s1, s2):
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [39]:
def run_iou_all():
    iou = {}
    for tune_i in annotated_ids:
        pats_i = get_pats(d[tune_i])
        for tune_j in links[tune_i]:
            iou[(tune_i, tune_j)] = intersection_over_union(pats_i, get_pats(d[tune_j]))
    return iou


# as before we comment-out a long-running computation:
# iou = run_iou_all()

# after running that, we would save the IOU data to disk
# f = open('data/thesession_annotated_omnisia_iou.pkl', 'wb')
# pickle.dump(iou, f)


In [55]:
# alternative: in case you want to avoid running IOU, which takes some time, 
# you can instead load a dump of the IOU results as below:
f = open('data/thesession_annotated_omnisia_iou.pkl', 'rb')
iou = pickle.load(f)

Now we are ready to produce results and look at performance. For each tune in the ground-truth tune family annotations, we rank all of the other tunes in The Session which have any patterns in common.
We rank them by IOU in decreasing order. Larger IOU values indicate higher similarity.

We simply count how many of the top 10 in that ranking are correct, ie are in the same tune family.

In [56]:
hit_results = {}
for tune_i in annotated_ids:
    sim = sorted([(iou[(tune_i, tune_j)], tune_j) for tune_j in links[tune_i]], reverse=True)
    # print(sim)
    hits = 0
    for simval, tune_j in sim[:10]:
        if tune_j in annotated_ids: # TODO is this correct?
            hits += 1
    hit_results[tune_i] = hits / 10

We see results are usually 0.0-0.3, ie 0-3 out of the top 10.

In [57]:
hit_results

{1029: 0.0,
 14252: 0.1,
 25163: 0.0,
 27315: 0.1,
 27316: 0.0,
 27842: 0.1,
 29508: 0.0,
 39472: 0.0,
 75: 0.1,
 12552: 0.0,
 12553: 0.1,
 12554: 0.0,
 12556: 0.1,
 12557: 0.0,
 12558: 0.0,
 12559: 0.0,
 12560: 0.0,
 21434: 0.1,
 23677: 0.1,
 24548: 0.0,
 24549: 0.0,
 24681: 0.3,
 25452: 0.0,
 25695: 0.1,
 26044: 0.0,
 26249: 0.0,
 29406: 0.1,
 31921: 0.0,
 35686: 0.1,
 42137: 0.0,
 29348: 0.1,
 29515: 0.1,
 31916: 0.0,
 32702: 0.0,
 1805: 0.2,
 27795: 0.0,
 33355: 0.0,
 33356: 0.1,
 36399: 0.0,
 38698: 0.0,
 2479: 0.1,
 15787: 0.0,
 35452: 0.0,
 1104: 0.1,
 1996: 0.0,
 4101: 0.1,
 4508: 0.0,
 14355: 0.0,
 14356: 0.1,
 14357: 0.1,
 15415: 0.0,
 17110: 0.0,
 17111: 0.0,
 17112: 0.0,
 24050: 0.0,
 26494: 0.0,
 27198: 0.0,
 28174: 0.0,
 28821: 0.1,
 31201: 0.0,
 32399: 0.0,
 34225: 0.0,
 36690: 0.1,
 39782: 0.0,
 27066: 0.0,
 249: 0.0,
 250: 0.2,
 12965: 0.0,
 12966: 0.2,
 12967: 0.1,
 12968: 0.1,
 12969: 0.2,
 12970: 0.1,
 12971: 0.0,
 12972: 0.2,
 21776: 0.0,
 25089: 0.2,
 27843: 0.2,


In [48]:
#f = open('data/thesession_annotated_omnisia_precision10_results.pkl', 'wb')
#pickle.dump(hit_results, f)

We can compare the results (in text format, above) to the results achieved by Diamond (reported in Polifonia deliverables D5.5-5.6 and MSc thesis, 2024). Each result is effectively the number of 'hits' in the top 10 predictions, for a given query tune. As shown in the image below, the 'motif' method of Diamond sometimes gives a low result - 0 to 3 out of 10 - but often achieves values above 5 out of 10, never achieved using SIATEC patterns.

![](img/motif_map_per_tune.png)

### Discussion and Conclusion

Meredith (2013) writes:
"Both algorithms [COSIATEC and SIATECCompress] are founded on the hypothesis that the
best ways of understanding a piece of music are those that
are represented by the shortest descriptions of the piece. In
other words, they are designed to explore the notion that
music analysis is effectively just music compression."

This hypothesis is central to my (McDermott) view of music analysis also. I look to papers such as Schmidhuber's *Low-Complexity Art*. In related work (working with Maziar Kanani and Seán O'Leary, not part of the Polifonia project), I use the *Assembly Pathway* (Cronin and colleagues) and the *Sequitur* algorithm (Nevil-Manning and Witten).

However, it seems that a compression approach is not necessarily well-suited to the task of *linking pieces of music*. This difference in goals likely explains the unexpected results.
